In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../datasets/autoria/autoria_clean_data.csv')
orig_df = df.copy()

In [3]:
df.head()

,id,brand,price_USD,mileage_kkm,fuel_type,transmission_type,pub_date,year_made,model,engine_size
0,0,Mercedes-Benz,8999,159,gas/petrol,automatic,2021-05-21,2007,E 280 4MATIC AT CLASSIC,3.0
1,1,Mercedes-Benz,79900,103,diesel,automatic,2021-05-15,2015,G 350 Designo,3.0
2,2,BMW,15999,145,petrol,automatic,2021-04-18,2010,550,4.4
3,3,MINI,10200,111,petrol,automatic,2021-05-21,2014,Countryman S,1.6
4,4,Nissan,13900,97,petrol,automatic,2021-05-19,2010,X-Trail,2.0


In [4]:
# Selecting features and output
X = df[['mileage_kkm', 'year_made', 'engine_size']].head(400)
y = df['price_USD'].head(400)

In [5]:
# DIviding data into training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = np.expand_dims(X_train, 1)
X_test = np.expand_dims(X_test, 1)

X_train, y_train = (np.array(X_train), np.array(y_train))
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))

X_test, y_test = (np.array(X_test), np.array(y_test))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

print(X_train.shape)
print(y_train.shape)
print('---')
print(X_test.shape)
print(y_test.shape)

(268, 1, 3)
(268,)
---
(132, 1, 3)
(132,)


In [6]:
regressior = Sequential()

regressior.add(LSTM(4, input_shape = (X_train.shape[1], X_train.shape[2])))
regressior.add(Dense(1))


In [7]:
regressior.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
regressior.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test), verbose=1)

Epoch 1/100
17/17 [==============================] - 1s 25ms/step - loss: nan - val_loss: nan
Epoch 2/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 3/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 4/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 6/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 7/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 8/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 9/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 11/100
17/17 [==============================] - 0s 2ms/step - 

17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 89/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 90/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 91/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 92/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 93/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 94/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 95/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 96/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 97/100
17/17 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 98/100
17/17 [==============================] - 0s 2ms/step - loss:

In [9]:
regressior.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 4)                 128       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 133
Trainable params: 133
Non-trainable params: 0
_________________________________________________________________


In [10]:
y_pred = regressior.predict(X_test)
y_pred

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      